In [1]:
# 패키지 import
import torch
import torch.nn as nn
import torch.nn.functional as F

# PSPNet 네트워크 구성 및 구현

In [2]:
class PSPNet(nn.Module):
    def __init__(self, n_classes):
        super(PSPNet, self).__init__()

        block_config = [3,4,6,3]
        img_size = 475
        img_size_8 = 60

        self.feature_conv = FeatureMap_convolution()
        self.feature_res_1 = ResidualBlockPSP(n_blocks=block_config[0], in_channels=128, mid_channels=64,out_channels=256, stride=1, dilation=1)
        self.feature_res_2 = ResidualBlockPSP(n_blocks=block_config[1], in_channels=256, mid_channels=128,out_channels=512, stride=2, dilation=1)
        self.feature_dilated_res_1 = ResidualBlockPSP(n_blocks=block_config[2], in_channels=512, mid_channels=256,out_channels=1024, stride=1, dilation=2)
        self.feature_dilated_res_2 = ResidualBlockPSP(n_blocks=block_config[3], in_channels=1024, mid_channels=512,out_channels=2048, stride=1, dilation=4)

        self.pyramid_pooling = PyramidPooling(in_channels=2048, pool_size=[6,3,2,1], height=img_size_8, width=img_size_8)
        
        self.decode_feature = DecodePSPFeature(height=img_size, width=img_size, n_classes=n_classes)

        self.aux = AuxiliaryPSPLayers(in_channels=1024, height=img_size, width=img_size, n_classes=n_classes)

    def forward(self, x):
        x = self.feature_conv(x)
        x = self.feature_res_1(x)
        x = self.feature_res_2(x)
        x = self.feature_dilated_res_1(x)

        output_aux = self.aux(x)

        x = self.feature_dilated_res_2(x)

        x = self.pyramid_pooling(x)
        output = self.decode_feature(x)

        return (output, output_aux)



## FeatureMap_convolution 구현

In [3]:
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride,padding, dilation, bias):
        super(conv2DBatchNormRelu,self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,stride,padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        outputs = self.relu(x)
        return outputs

In [4]:
class FeatureMap_convolution(nn.Module):
    def __init__(self):
        super(FeatureMap_convolution, self).__init__()

        self.cbnr_1 = conv2DBatchNormRelu(in_channels=3,out_channels=64,kernel_size=3,stride=2,padding=1, dilation=1, bias=False)
        self.cbnr_2 = conv2DBatchNormRelu(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1,dilation=1, bias=False)
        self.cbnr_3 = conv2DBatchNormRelu(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1,dilation=1, bias=False)

        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.cbnr_1(x)
        x = self.cbnr_2(x)
        x = self.cbnr_3(x)

        outputs = self.max_pool(x)
        return outputs

## ResidualBlockPSP 구현

In [ ]:
class ResidualBlockPSP(nn.Sequential):
    def __init__(self, n_blocks, in_channels, mid_channels, out_channels, stride, dilation):
        super(ResidualBlockPSP, self).__init__()

        self.add_module = ("block1", bottleNeckPSP(in_channels, mid_channels,out_channels,stride, dilation))

        for i in range(n_blocks+1):
            self.add_module("block"+str(i+2), bottleNeckIdentifyPSP(out_channels, mid_channels, stride, dilation))

## bottleNeckPSP, bottleNeckIdentifyPSP

In [6]:
class conv2DBatchNorm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNorm, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        outputs = self.batchnorm(x)
        return outputs

class bottleNeckPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride, dilation):
        super(bottleNeckPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.cbr_3 = conv2DBatchNormRelu(mid_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.cb_residual = conv2DBatchNorm(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, dilation=1, biase=False)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cbr_3(self.cbr_2(self.cbr1(x)))
        residual = self.cb_residual(x)
        return self.relu(conv + residual)

class bottleNectIdentifyPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, stride, dilation):
        super(bottleNectIdentifyPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(mid_channels, mid_channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)
        self.cbr_3 = conv2DBatchNormRelu(mid_channels, in_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cbr_3(self.cbr_2(self.cbr_2(x)))
        residual = x
        return self.relu(conv + residual)

# Pyramid Pooling 모듈  설명 및 구현